In [7]:
import torch
from PIL import Image
import os                       # for working with files
import torchvision.transforms as transforms   # for transforming images into tensors 
from torchvision import datasets,transforms,models   # for working with classes and images
import pandas as pd
import torch.nn.functional as nnf
from config import class_names

In [8]:
model = torch.load('Model/resnet18_full.pth')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [10]:
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result


In [11]:
image = Image.open("Test_folder/1_4.png")
image = expand2square(image, (256, 256, 256))
image = image.convert('RGB')
image.save('Test_folder/after_out.jpg', quality=95)
image_tensor = transform(image).unsqueeze(0)
image_tensor=image_tensor.to("cuda")
with torch.no_grad():
    output = model(image_tensor)
    predicted_class = torch.softmax(output, dim=1)
    y_pred = torch.softmax(output, dim=1).argmax(dim=1)
    top_p, top_class = predicted_class.topk(37, dim = 1)
    print(class_names[y_pred])
# Example output

# Mapping indices to class labels
predicted_labels = [class_names[i] for i in top_class[0]]
probabilities = [round(float(p), 5) for p in top_p[0]]

output_zip=zip(predicted_labels,probabilities)

df = pd.DataFrame({"Class": predicted_labels, "Probability": probabilities})

# Sorting the DataFrame by probability in descending order
df = df.sort_values(by="Probability", ascending=False).reset_index(drop=True)
df.index = df.index + 1
print(df)

Potato___Early_blight
                                                Class  Probability
1                               Potato___Early_blight      0.99456
2                                Potato___Late_blight      0.00446
3                                    Potato___healthy      0.00059
4                                 Mango___Sooty Mould      0.00016
5                                  Mango___Gall Midge      0.00007
6                                    Rice___LeafBlast      0.00003
7                                   Grape___Black_rot      0.00003
8                                      Rice___healthy      0.00003
9                                        Rice___Hispa      0.00001
10                                    Mango___healthy      0.00001
11                             Mango___Powdery Mildew      0.00001
12         Grape___Leaf_blight_(Isariopsis_Leaf_Spot)      0.00001
13                              Tomato___Early_blight      0.00001
14                                Mango_